In [ ]:
 import pandas as pd
import numpy as np

train_temp = [] 
test_temp = []
#共有18種汙染物,先做出18個容器作為分類用
for i in range(18):
    train_temp.append([])
    test_temp.append([])

temp = pd.read_excel('107年新竹站_20190315.xlsx')
"""
資料前處理
"""    

'''
以下是將特殊符號或是空值用0替代，但本次作業用前後兩筆資料取平均代替

temp.replace(['[-+]?([0-9]*[\.]?[0-9]+[#*x])'] ,0 , regex=True , inplace=True) 
#正規表達式 +*? 取特殊符號(#*x)的資料replace為0
temp = temp.fillna(0) 
#空值以0取代
'''

temp = temp.replace("NR",0) 
#NR表示無降雨，以0替代

"""
時間序列shift用
"""        
def shift(l, n): 
    return l[n:]

#l為list名稱,n為移動多少
#利用list性質,做出該效果
#eg. l=[0,1,2,3,4]  
#=l=shift(l,2)轉換為由2開始 l=[2,3,4],達成時間往前推移兩格的效果(由第2個開始取)
#l=shift(l,-1) 轉換為由倒數第一個開始取 l=[4]


In [ ]:
"""
將資料以時間分成train跟test資料
#train = temp[temp["日期"].between('2018/10/01','2018/11/30')]                
#test = temp[temp["日期"] >= '2018/12/01']

train data 2個月，2018/10/01到2018/11/30
test data 1個月，2018/12/01-2018/12/31
將train及test data中的汙染物數值用時序連接在一起
例如：一列即為10/1-11/30的所有資料
"""

#,行---列|||,i行j列
for i in range(0,len(temp)):
    if temp.iloc[i,0] >= '2018/10/01' and temp.iloc[i,0] <= '2018/11/30': #由i(行)的第0列開始搜尋符合條件的資料,也就是每行的0列
        #第3-27列是汙染物值,取3-27列
        for j in range(3,27): 
            train_temp[i%18].append((temp.iloc[i,j]))#[i%18]是把資料分成18堆(時序資料會自動分類),但為啥結果是從第7種汙染源開始?
    elif temp.iloc[i,0] >= '2018/12/01':
        for j in range(3,27):
            test_temp[i%18].append((temp.iloc[i,j]))

In [ ]:
"""
train data，只拿PM2.5值
"""

#第4行的汙染物是PM2.5 (由NOx開始)
#len(train_PM25_temp) 
train_PM25_temp =  train_temp[3]

#處理異常值及空值，將其轉為前後項的平均
for i in range(len(train_PM25_temp)):
    #處理異常值及空值，將其轉為前後項的平均
    j = i-1
    k = i+1
    #使用find函數:若找到目標則回傳目標物,都沒找到則回傳-1
    #當有i是異常值:
    if str(train_PM25_temp[i]).find("*") != -1 or str(train_PM25_temp[i]).find("#") != -1 or str(train_PM25_temp[i]).find("A") != -1 or np.isnan(train_PM25_temp[i]) == True:
          #若i的前項(j)是異常值,就用j的前一項(j-1)替代j
          while str(train_PM25_temp[j]).find("*") != -1 or str(train_PM25_temp[j]).find("#") != -1 or str(train_PM25_temp[j]).find("A") != -1 or np.isnan(train_PM25_temp[j]) == True:
                   j = j-1
          #若i的後項(k)是異常值,就用k的後一項(k+1)替代k
          while str(train_PM25_temp[k]).find("*") != -1 or str(train_PM25_temp[k]).find("#") != -1 or str(train_PM25_temp[k]).find("A") != -1 or np.isnan(train_PM25_temp[k]) == True:
                   k = k+1
          #確認沒問題後做平均處理得到i
          train_PM25_temp[i] = (train_PM25_temp[j] + train_PM25_temp[k]) / 2
train_X = []
train_y = []
period = 6
#用六小時的資料去預測第七小時的PM2.5
length = len(train_PM25_temp) - period
for i in range(length):
    if len(train_PM25_temp) > period:
        train_X.append(train_PM25_temp[0:period]) #x為1-6小時的資料
        train_y.append(train_PM25_temp[6]) #y為第7小時的資料(也就是正確答案)
        train_PM25_temp = shift(train_PM25_temp,1)
        #用shift把資料往前補，方便固定前0-5欄位都是x,第6欄是y 

In [ ]:
"""
#test data，只拿PM2.5值
"""
#第4個欄位是PM2.5
test_PM25_temp =  test_temp[3]#抽取pm2.5的部分做檢驗(為第三個list)
#異常值處理同上
for i in range(len(test_PM25_temp)):
    j = i-1
    k = i+1
    if str(test_PM25_temp[i]).find("*") != -1 or str(test_PM25_temp[i]).find("#") != -1 or str(test_PM25_temp[i]).find("A") != -1 or np.isnan(test_PM25_temp[i]) == True:
          while str(test_PM25_temp[j]).find("*") != -1 or str(test_PM25_temp[j]).find("#") != -1 or str(test_PM25_temp[j]).find("A") != -1 or np.isnan(test_PM25_temp[j]) == True:
                   j = j-1
          while str(test_PM25_temp[k]).find("*") != -1 or str(test_PM25_temp[k]).find("#") != -1 or str(test_PM25_temp[k]).find("A") != -1 or np.isnan(test_PM25_temp[k]) == True:
                   k = k+1
          test_PM25_temp[i] = (test_PM25_temp[j] + test_PM25_temp[k]) / 2
          
test_X = []
test_y = []
period = 6
length = len(test_PM25_temp) - period
#同上
for i in range(length):
    if len(test_PM25_temp) > period:
        test_X.append(test_PM25_temp[0:period])
        test_y.append(test_PM25_temp[6])
        test_PM25_temp = shift(test_PM25_temp,1)

##linear regression復刻版-簡單linear
[來源](https://www.twblogs.net/a/5bd33c3c2b717778ac1ff843

In [ ]:
#前兩行==>生成數個b截距與w權重,並以他們做不同組合求最小loss

'''
#0.0855=171/2000,原為使用-+0.0855step0.00005生成3420個b的值(反推為171/2000*x*2=3420,x=20000,step=1/20000)
#3420怎麼來的(228天*15組=3420組數據)
思路：利用前9個小時的PM2.5的數據來預測下一個小時的PM2.5數據，這樣一天的數據可以劃分爲15組，去除掉PM2.5小於0的無效觀測天數據
由於我們有做資料前處理,皆為有效數據,2018/10/01到2018/11/30有61天*17=1037組
#應使用(-0.25925, 0.25925, step=0.0005)(跑不動我縮小作者倍率模擬一下流程)

#0.07695=1539/20000,原為使用step0.000005生成30780組w的值，每組w有9個值w0-w8
#改成1037組(一組6個權重w),(-0.15555, 0.15555, step=0.0005)
'''

X = np.arange(-0.0855, 0.0855, step=0.005)
Y = np.arange(-0.07695, 0.07695, step=0.0005)

Loss = np.zeros((len(X), len(X)))
# z是一個值全爲0，len(X)行len(X)列的矩陣,即3420組w*3420組b的不同組合，每種組合都計算一下它們的loss值（樣本爲全體數據）
for i in range(len(X)):
	for j in range(len(X)):
		# 雙層for循環，手動計算方差和的平均值
		b = X[i] #x[0],x[1]...
		w = Y[j * 6:j * 6 + 6] #y[0:6],y[1,7]...
		Loss[i][j] = 0
		for index, x_temp in enumerate(train_X):
            # 計算每種w和b的組合的loss值,結果y和預測值的平方差
			Loss[i][j] = Loss[i][j] + (
					train_y[index] - b - w[0] * x_temp[0] - w[1] * x_temp[1] - w[2] *
					x_temp[2] - w[3] * x_temp[3] - w[4] * x_temp[4]- w[5] * x_temp[5]) ** 2
#方差和的平均值
Loss[i][j] = Loss[i][j] / len(train_X)
print(Loss[i][j]) #這就是mae吧??
print(b, w)


352.070673538176
0.08450000000000014 [0.02505 0.02555 0.02605 0.02655 0.02705 0.02755]


##使用BGD的linear

---
批量梯度下降:一次使用全部樣本來更新w跟b(參數)

蜥蜴書p119


In [ ]:
import matplotlib.pyplot as plt
# 定義loss函數,使用函數可以不斷自動更新(像是loop中loop的感覺?)
# xdata即輸入的x訓練數據組，ydata即輸入的y訓練數據組，wdata即設定的w0-w8，bdata即設定的b
def loss_function1(xdata, ydata, wdata, bdata):
	loss = 0
	for n, x in enumerate(xdata):
		# 計算loss
		loss = loss + (
				ydata[n] - bdata - wdata[0] * x[0] - wdata[1] * x[1] - wdata[2] *
				x[2] - wdata[3] * x[3] - wdata[4] * x[4]
				- wdata[5] * x[5]) ** 2
	return loss / len(xdata)




# 定義b的初始值 
b = -0.08542942192781317
# 定義w0-w5的初始值 
w = [-0.33030855334867937, 0.05388424505307146, 0.5966401292268716, -0.6020419118650027, -0.28417190443282037,
	 1.3284417881100619]

# 定義爲學習速率lr的初始值
# lr的值設置可以參照我們模擬手動計算時設置的步長
lrw, lrb = 0.0000000001, 0.000000001
# 定義訓練次數iteration
iteration = 1000
# 定義b_history、w_history、loss_history用來存儲訓練過程中更新的w、b、和loss
b_history = [b]
w_history = [w]
# loss_history初始值需要計算，用初始的w和b輸入函數loss_function1進行計算
loss_history = [loss_function1(train_X, train_y, w, b)]

# 定義w偏導數和b的偏導數初始值爲0
b_grad = 0.0
w_grad = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


# 定義更新w的偏導數、b的偏導數、w和b的函數 
# w_grad即w的偏導數，b_grad即b的偏導數,xdata即輸入的x訓練數據組，ydata即輸入的y訓練數據組，wdata即設定的w0-w8，bdata即設定的b

def update_b_grad_and_w_grad_and_w_and_b(bgrad, wgrad, xdata, ydata, wdata, bdata):
	# xdata的數據組規模數量表示了我們用多大規模的數據來更新一次w和b的偏導數，這就是BGD、SGD、MBGD之間的區別
	for n, x in enumerate(xdata):
		bgrad = bgrad - 2.0 * (
				ydata[n] - bdata - wdata[0] * x[0] - wdata[1] * x[1] - wdata[2] * x[2] - wdata[3] * x[3] - wdata[4] *
				x[4] - wdata[5] * x[5] ) * 1.0
		for m in range(6):
			wgrad[m] = wgrad[m] - 2.0 * (
					ydata[n] - bdata - wdata[0] * x[0] - wdata[1] * x[1] - wdata[2] * x[2] - wdata[3] * x[3] - wdata[
				4] *x[4] - wdata[5] * x[5]) * x[m]
	# 用偏導數bgrad/wgrad乘以學習速率lr來更新w和b
	bdata = bdata - lrb * bgrad
	for m in range(6):	
		wdata[m] = wdata[m] - lrw * w_grad[m]
	return bdata, wdata


# 存儲訓練得出的最好的w和b值，並記錄此時的training error
b_train_best = 1
w_train_best = []
training_error = 10000
'''
每訓練一次存儲更新的w和b
# 並且將更新的w和b的值賦值給w和b變量
# 也就是說每訓練一次更新了4個變量：b_grad、w_grad、w、b，b_grad和w_grad直接在函數體內更新，w和b要記錄更新後的值，所以作爲函數返回值
# 在下面這句更新
'''
for i in range(iteration):
	b_temp, w_temp = update_b_grad_and_w_grad_and_w_and_b(b_grad , w_grad, train_X, train_y, w, b)
	b, w = b_temp, w_temp #更新
	loss_history.append(loss_function1(train_X, train_y, w_temp, b_temp))
	if loss_history[-1] < training_error:
		training_error = loss_history[-1]
		b_train_best = b_temp
		w_train_best.clear()
		[w_train_best.append(i) for i in w_temp]
	b_history.append(b_temp)
	w_history.append(w_temp)
# 得到訓練出來的最佳w、b、Loss
print(training_error, b_train_best, w_train_best)
testing_error = loss_function1(test_X, test_y, w_train_best, b_train_best)
print("BGDlinear的MAE是{}".format(testing_error))#mae


10.936221935727028 -0.08432999764210623 [0.06314656268580168, -0.08131408805172351, 0.32185961671058944, -0.24839935799261728, -0.2325949043397792, 1.1697895918974865]
BGDlinear的MAE是12.64822230345141


##使用SGD的linear


---
隨機梯度下降:一次使用一個樣本更新w跟b(有m個樣本就會迭代m次)

蜥蜴書p121


In [ ]:
def loss_function1(xdata, ydata, wdata, bdata):
	loss = 0
	loss = loss + (
			ydata - bdata - wdata[0] * xdata[0] - wdata[1] * xdata[1] - wdata[2] *
			xdata[2] - wdata[3] * xdata[3] - wdata[4] * xdata[4]
			- wdata[5] * xdata[5]) ** 2
	return loss


# 函數返回方差和的平均值

 
# 定義b的初始值
b = -0.08287711928778152
# 定義w0-w8的初始值
w = [0.2336750995700671, 0.0011688047729486227, -0.07906800252618741, -0.2912006071443291, 0.23385921935943532,
	 0.8659526367683459]
# 注意上面的初始值是我們模擬手動計算時找到的一組Loss值較小的值，比如這組的Loss值爲235.5967253011724
# 定義爲學習速率lr的初始值
# lr的值設置可以參照我們模擬手動計算時設置的步長
lrw, lrb = 0.0000000001, 0.000000001
# 定義訓練次數iteration
iteration = 1000
# 定義b_history、w_history、loss_history用來存儲訓練過程中更新的w、b、和loss
b_history = [b]
w_history = [w]
# loss_history初始值需要計算，用初始的w和b輸入函數loss_function1進行計算
loss_history = []

# 定義w偏導數和b的偏導數初始值爲0
b_grad = 0.0
w_grad = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


# 定義更新w的偏導數、b的偏導數、w和b的函數
# bgrad即w的偏導數，bgrad即b的偏導數，xdata即輸入的x訓練數據組，ydata即輸入的y訓練數據組，wdata即設定的w0-w8，bdata即設定的b
def update_b_grad_and_w_grad_and_w_and_b(bgrad, wgrad, xdata, ydata, wdata, bdata):
	# 定義SGD方法的梯度下降優化器
	bgrad = bgrad - 2.0 * (
			ydata - bdata - wdata[0] * xdata[0] - wdata[1] * xdata[1] - wdata[2] * xdata[2] - wdata[3] * xdata[3] -
			wdata[4] *
			xdata[4] - wdata[5] * xdata[5]) * 1.0
	for m in range(6):
		wgrad[m] = wgrad[m] - 2.0 * (
				ydata - bdata - wdata[0] * xdata[0] - wdata[1] * xdata[1] - wdata[2] * xdata[2] - wdata[3] * xdata[3] -
				wdata[4] * xdata[4] - wdata[5] * xdata[5]) * xdata[m]
	# 用偏導數乘以學習速率來更新w和b
	bdata = bdata - lrb * bgrad
	for m in range(6):
		wdata[m] = wdata[m] - lrw * w_grad[m]
	return bdata, wdata


# 存儲訓練得出的最好的w和b值，並記錄此時的training error
b_train_best = 0
w_train_best = []
training_error = 10000

for i in range(iteration):
	x_data_SGD = train_X[i]
	y_data_SGD = train_y[i]
	b_temp, w_temp = update_b_grad_and_w_grad_and_w_and_b(b_grad, w_grad, x_data_SGD, y_data_SGD, w, b)
	# 每訓練一次存儲更新的w和b
	# 並且將更新的w和b的值賦值給w和b變量
	# 也就是說每訓練一次更新了4個變量：b_grad、w_grad、w、b，b_grad和w_grad直接在函數體內更新，w和b要記錄更新後的值，所以作爲函數返回值
	# 在下面這句更新
	b, w = b_temp, w_temp
	loss_history.append(loss_function1(x_data_SGD, y_data_SGD, w_temp, b_temp))
	if loss_history[-1] < training_error:
		training_error = loss_history[-1]
		b_train_best = b_temp
		w_train_best.clear()
		[w_train_best.append(i) for i in w_temp]
	b_history.append(b_temp)
	w_history.append(w_temp)
# 得到訓練出來的最佳w、b、Loss
print(training_error, b_train_best, w_train_best)
testing_error = 0
for i in range(len(test_X)):
	testing_error = testing_error + loss_function1(test_X[i], test_y[i], w_train_best, b_train_best)
testing_error = testing_error / len(test_X)
print("SGDlinear的MAE是{}".format(testing_error))

1.0620307248927059e-05 -0.08287648265235675 [0.23386181575939752, 0.0013839533648000496, -0.07879383268728844, -0.29090315395689237, 0.23410200731620903, 0.8662277079908691]
SGDlinear的MAE是17.29715712586813


##使用MBGD的linear

---
小批量梯度下降 :一次使用固定量的樣本數更新b跟w(不是使用全部樣本)

蜥蜴書p124



In [ ]:
def loss_function1(xdata, ydata, wdata, bdata):
	loss = 0
	for n, x in enumerate(xdata):
		# 計算loss
		loss = loss + (
				ydata[n] - bdata - wdata[0] * x[0] - wdata[1] * x[1] - wdata[2] *
				x[2] - wdata[3] * x[3] - wdata[4] * x[4]
				- wdata[5] * x[5]) ** 2
	return loss / len(xdata)


# 定義b的初始值
b = -0.08287711928778152
# 定義w0-w5的初始值
w = [0.2336750995700671, 0.0011688047729486227, -0.07906800252618741, -0.2912006071443291, 0.23385921935943532,
	 0.8659526367683459]
# 注意上面的初始值是我們模擬手動計算時找到的一組Loss值較小的值，比如這組的Loss值爲235.5967253011724
# 定義爲學習速率lr的初始值
# lr的值設置可以參照我們模擬手動計算時設置的步長
lrw, lrb = 0.0000000001, 0.000000001
# 定義訓練次數iteration
iteration = 1000
# 定義b_history、w_history、loss_history用來存儲訓練過程中更新的w、b、和loss
b_history = [b]
w_history = [w]
# loss_history初始值需要計算，用初始的w和b輸入函數loss_function1進行計算
loss_history = []

# 定義w偏導數和b的偏導數初始值爲0
b_grad = 0.0
w_grad = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


# 定義更新w的偏導數、b的偏導數、w和b的函數
# bgrad即w的偏導數，bgrad即b的偏導數，xdata即輸入的x訓練數據組，ydata即輸入的y訓練數據組，wdata即設定的w0-w8，bdata即設定的b
def update_b_grad_and_w_grad_and_w_and_b(bgrad, wgrad, xdata, ydata, wdata, bdata):
	# xdata的數據組規模數量表示了我們用多大規模的數據來更新一次w和b的偏導數，這就是BGD、SGD、MBGD之間的區別
	for n, x in enumerate(xdata):
		bgrad = bgrad - 2.0 * (
				ydata[n] - bdata - wdata[0] * x[0] - wdata[1] * x[1] - wdata[2] * x[2] - wdata[3] * x[3] - wdata[4] *
				x[4] - wdata[5] * x[5]) * 1.0
		for m in range(6):
			wgrad[m] = wgrad[m] - 2.0 * (
					ydata[n] - bdata - wdata[0] * x[0] - wdata[1] * x[1] - wdata[2] * x[2] - wdata[3] * x[3] - wdata[
				4] *
					x[4] - wdata[5] * x[5]) * x[m]
	# 用偏導數乘以學習速率來更新w和b
	bdata = bdata - lrb * bgrad
	for m in range(6):
		wdata[m] = wdata[m] - lrw * w_grad[m]
	return bdata, wdata


# 存儲訓練得出的最好的w和b值，並記錄此時的training error
b_train_best = 1
w_train_best = []
training_error = 10000

for i in range(iteration):
	x_train_minibatch = train_X[0:200]
	y_train_minibatch = train_y[0:200]
	b_temp, w_temp = update_b_grad_and_w_grad_and_w_and_b(b_grad, w_grad, x_train_minibatch, y_train_minibatch, w, b)
	# 每訓練一次存儲更新的w和b
	# 並且將更新的w和b的值賦值給w和b變量
	# 也就是說每訓練一次更新了4個變量：b_grad、w_grad、w、b，b_grad和w_grad直接在函數體內更新，w和b要記錄更新後的值，所以作爲函數返回值
	# 在下面這句更新
	b, w = b_temp, w_temp
	loss_history.append(loss_function1(x_train_minibatch, y_train_minibatch, w_temp, b_temp))
	if loss_history[-1] < training_error:
		training_error = loss_history[-1]
		b_train_best = b_temp
		w_train_best.clear()
		[w_train_best.append(i) for i in w_temp]
	b_history.append(b_temp)
	w_history.append(w_temp)
# 得到訓練出來的最佳w、b、Loss
print(training_error, b_train_best, w_train_best)

print("MBGDlinear的MAE是{}".format(loss_function1(test_X, test_y, w_train_best, b_train_best)))#mae

7.855621448654821 -0.08280665925413777 [0.20961764415134249, -0.01298784752729445, -0.05832356771125835, -0.257609569255654, 0.235177384366057, 0.8870157251734297]
MBGDlinear的MAE是16.331761524603248


##心得



*  小結:MBGD的效果與BGD較相近,而且可以跑比較快


*   要熟悉演算法套件還有他們的特性
*   多練習迴圈&python
*  想想看前處理還可以怎麼做(大學問)



---

> 再試看看不同方法!

